___

<a href='http://www.pieriandata.com'> <img src='../Pierian_Data_Logo.png' /></a>
___

# Non-Negative Matric Factorization

Let's repeat thet opic modeling task from the previous lecture, but this time, we will use NMF instead of LDA.

## Data

We will be using articles scraped from NPR (National Public Radio), obtained from their website [www.npr.org](http://www.npr.org)

In [0]:
import pandas as pd

In [0]:
npr = pd.read_csv('/dbfs/FileStore/ds_data_files/npr.csv')

In [0]:
npr.head()

,Article
0,"In the Washington of 2016, even when the polic..."
1,Donald Trump has used Twitter — his prefe...
2,Donald Trump is unabashedly praising Russian...
3,"Updated at 2:50 p. m. ET, Russian President Vl..."
4,"From photography, illustration and video, to d..."


Notice how we don't have the topic of the articles! Let's use NMF to attempt to figure out clusters of the articles.

LDA processes text using count vectorizer using probabilities
NMF processes text using TFIDF due to the fact that it uses coefficients

## Preprocessing

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer

**`max_df`**` : float in range [0.0, 1.0] or int, default=1.0`<br>
When building the vocabulary ignore terms that have a document frequency strictly higher than the given threshold (corpus-specific stop words). If float, the parameter represents a proportion of documents, integer absolute counts. This parameter is ignored if vocabulary is not None.

**`min_df`**` : float in range [0.0, 1.0] or int, default=1`<br>
When building the vocabulary ignore terms that have a document frequency strictly lower than the given threshold. This value is also called cut-off in the literature. If float, the parameter represents a proportion of documents, integer absolute counts. This parameter is ignored if vocabulary is not None.

In [0]:
tfidf = TfidfVectorizer(max_df=0.95, min_df=2, stop_words='english')

In [0]:
dtm = tfidf.fit_transform(npr['Article'])

In [0]:
dtm

Out[8]: <11992x54777 sparse matrix of type '<class 'numpy.float64'>'
	with 3033388 stored elements in Compressed Sparse Row format>

## NMF

In [0]:
from sklearn.decomposition import NMF

In [0]:
nmf_model = NMF(n_components=7,random_state=42)

In [0]:
# This can take awhile, we're dealing with a large amount of documents! NMF performs faster than LDA
nmf_model.fit(dtm)

/databricks/python/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:312: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(("The 'init' value, when 'init=None' and "
Out[11]: NMF(n_components=7, random_state=42)

## Displaying Topics

In [0]:
len(tfidf.get_feature_names())

Out[12]: 54777

In [0]:
import random

for i in range(10):
    random_word_id = random.randint(0,54776)
    print(tfidf.get_feature_names()[random_word_id])

leave
marsalis
intermediary
warehouse
meh
genuflecting
swathes
productivity
executive
streetwalkers


In [0]:
for i in range(10):
    random_word_id = random.randint(0,54776)
    print(tfidf.get_feature_names()[random_word_id])

suleyman
strad
ptsd
aspiration
cripple
rearview
384
disinfection
meager
hardscrabble


In [0]:
len(nmf_model.components_)

Out[15]: 7

In [0]:
nmf_model.components_  # not probabilities any more but coefficients

Out[16]: array([[0.00000000e+00, 2.49950821e-01, 0.00000000e+00, ...,
        1.70313822e-03, 2.37544362e-04, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 8.22048918e-02, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [0.00000000e+00, 3.12379960e-02, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [5.89723338e-03, 0.00000000e+00, 1.50186440e-03, ...,
        7.06428924e-04, 5.85500542e-04, 6.89536542e-04],
       [4.01763234e-03, 5.31643833e-02, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [0]:
len(nmf_model.components_[0])

54777

In [0]:
single_topic = nmf_model.components_[0]

In [0]:
# Returns the indices that would sort this array.
single_topic.argsort()

array([    0, 27208, 27206, ..., 36283, 54692, 42993], dtype=int64)

In [0]:
# Word least representative of this topic
single_topic[18302]

0.0

In [0]:
# Word most representative of this topic
single_topic[42993]

2.005055165418576

In [0]:
# Top 10 words for this topic:
single_topic.argsort()[-10:]

array([10421, 31464,     1, 54403, 28659, 54412, 36310, 33390, 36283,
       42993], dtype=int64)

In [0]:
top_word_indices = single_topic.argsort()[-10:]

In [0]:
for index in top_word_indices:
    print(tfidf.get_feature_names()[index])

disease
percent
women
virus
study
water
food
people
zika
says


These look like business articles perhaps... Let's confirm by using .transform() on our vectorized articles to attach a label number. But first, let's view all the 10 topics found.

In [0]:
for index,topic in enumerate(nmf_model.components_):
    print(f'THE TOP 15 WORDS FOR TOPIC #{index}')
    print([tfidf.get_feature_names()[i] for i in topic.argsort()[-15:]])
    print('\n')

THE TOP 15 WORDS FOR TOPIC #0
['new', 'research', 'like', 'patients', 'health', 'disease', 'percent', 'women', 'virus', 'study', 'water', 'food', 'people', 'zika', 'says']


THE TOP 15 WORDS FOR TOPIC #1
['gop', 'pence', 'presidential', 'russia', 'administration', 'election', 'republican', 'obama', 'white', 'house', 'donald', 'campaign', 'said', 'president', 'trump']


THE TOP 15 WORDS FOR TOPIC #2
['senate', 'house', 'people', 'act', 'law', 'tax', 'plan', 'republicans', 'affordable', 'obamacare', 'coverage', 'medicaid', 'insurance', 'care', 'health']


THE TOP 15 WORDS FOR TOPIC #3
['officers', 'syria', 'security', 'department', 'law', 'isis', 'russia', 'government', 'state', 'attack', 'president', 'reports', 'court', 'said', 'police']


THE TOP 15 WORDS FOR TOPIC #4
['primary', 'cruz', 'election', 'democrats', 'percent', 'party', 'delegates', 'vote', 'state', 'democratic', 'hillary', 'campaign', 'voters', 'sanders', 'clinton']


THE TOP 15 WORDS FOR TOPIC #5
['love', 've', 'don', 'al

### Attaching Discovered Topic Labels to Original Articles

In [0]:
dtm

Out[18]: <11992x54777 sparse matrix of type '<class 'numpy.float64'>'
	with 3033388 stored elements in Compressed Sparse Row format>

In [0]:
dtm.shape

Out[19]: (11992, 54777)

In [0]:
len(npr)

Out[20]: 11992

In [0]:
topic_results = nmf_model.transform(dtm)

In [0]:
topic_results.shape

Out[22]: (11992, 7)

In [0]:
topic_results[0]

Out[23]: array([0.        , 0.12075603, 0.00140297, 0.05919954, 0.01518909,
       0.        , 0.        ])

In [0]:
topic_results[0].round(2)

Out[24]: array([0.  , 0.12, 0.  , 0.06, 0.02, 0.  , 0.  ])

In [0]:
topic_results[0].argmax()

Out[25]: 1

This means that our model thinks that the first article belongs to topic #1.

### Combining with Original Data

In [0]:
npr.head()

,Article
0,"In the Washington of 2016, even when the polic..."
1,Donald Trump has used Twitter — his prefe...
2,Donald Trump is unabashedly praising Russian...
3,"Updated at 2:50 p. m. ET, Russian President Vl..."
4,"From photography, illustration and video, to d..."


In [0]:
topic_results.argmax(axis=1)

Out[27]: array([1, 1, 1, ..., 0, 4, 3])

In [0]:
npr['Topic'] = topic_results.argmax(axis=1)

In [0]:
npr.head(10)

,Article,Topic
0,"In the Washington of 2016, even when the polic...",1
1,Donald Trump has used Twitter — his prefe...,1
2,Donald Trump is unabashedly praising Russian...,1
3,"Updated at 2:50 p. m. ET, Russian President Vl...",3
4,"From photography, illustration and video, to d...",6
5,I did not want to join yoga class. I hated tho...,5
6,With a who has publicly supported the debunk...,0
7,"I was standing by the airport exit, debating w...",0
8,"If movies were trying to be more realistic, pe...",0
9,"Eighteen years ago, on New Year’s Eve, David F...",5


## Great work!